In [1]:
!nvidia-smi

Wed Nov 29 23:15:47 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070 L...    On | 00000000:01:00.0 Off |                  N/A |
| N/A   45C    P8               10W /  25W|    147MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import argparse, os
import numpy as np
from himalaya.backend import set_backend
from himalaya.ridge import RidgeCV
from himalaya.scoring import correlation_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import time
import torch
import torch.nn as nn
import torch.optim as optim


/home/aditya/anaconda3/envs/ldm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# python ridge.py --target c --roi ventral --subject subj01
# python ridge.py --target init_latent --roi early --subject subj01

In [4]:
target = 'c'
roi = 'ventral'
subject = 'subj01'
backend = set_backend("numpy", on_error="warn")

mridir = f'../../mrifeat/{subject}/'
featdir = '../../nsdfeat/subjfeat/'
savedir = f'../..//decoded_net/{subject}/'
os.makedirs(savedir, exist_ok=True)

In [5]:
# if target == 'c' or target == 'init_latent': # CVPR
#     alpha = [0.000001,0.00001,0.0001,0.001,0.01, 0.1, 1]
 
# For experimentations
if target == 'c' or target == 'init_latent': # CVPR
    alpha = [0.01]

In [6]:
X = []
X_te = []

for croi in [roi]:
    if 'conv' in target: # We use averaged features for GAN due to large number of dimension of features
        cX = np.load(f'{mridir}/{subject}_{croi}_betas_ave_tr.npy').astype("float32")
    else:
        cX = np.load(f'{mridir}/{subject}_{croi}_betas_tr.npy').astype("float32")
    
    cX_te = np.load(f'{mridir}/{subject}_{croi}_betas_ave_te.npy').astype("float32")
    
    X.append(cX)
    X_te.append(cX_te)

X = np.hstack(X)
X_te = np.hstack(X_te)


Y = np.load(f'{featdir}/{subject}_each_{target}_tr.npy').astype("float32").reshape([24980,-1])
Y_te = np.load(f'{featdir}/{subject}_ave_{target}_te.npy').astype("float32").reshape([X_te.shape[0],-1])

In [7]:
print(f'Now making decoding model for... {subject}:  {roi}, {target}')
print(f'X {X.shape}, Y {Y.shape}, X_te {X_te.shape}, Y_te {Y_te.shape}')

Now making decoding model for... subj01:  ventral, c
X (24980, 7604), Y (24980, 59136), X_te (982, 7604), Y_te (982, 59136)


______
_____

## Subset small data 

In [8]:
num_samples = 50
X_train = X[:num_samples, :].copy()
Y_train = Y[:num_samples, :].copy()
X_test = X_te[:num_samples//5, :].copy()
Y_test = Y_te[:num_samples//5, :].copy()

In [9]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((50, 7604), (50, 59136), (10, 7604), (10, 59136))

In [ ]:
# Calculate the mean and standard deviation along the rows (axis=1)
mean = np.mean(X_train, axis=0, keepdims=True)
std = np.std(X_train, axis=0, keepdims=True)

# Standardize the rows of X
X_train_std = (X_train - mean) / std

_______
_______

## Ridge Himalaya

In [10]:
ridge = RidgeCV(alphas=alpha,cv=5)

preprocess_pipeline = make_pipeline(
    StandardScaler(with_mean=True, with_std=True),
)

pipeline = make_pipeline(
    preprocess_pipeline,
    ridge,
)  

In [11]:
pipeline.fit(X_train, Y_train)

/home/aditya/anaconda3/envs/ldm/lib/python3.8/site-packages/himalaya/ridge/_random_search.py:491: UserWarning: Solving ridge is slower than solving kernel ridge when n_samples < n_features (here 50 < 7604). Using a linear kernel in himalaya.kernel_ridge.KernelRidgeCV or himalaya.kernel_ridge.solve_kernel_ridge_cv_eigenvalues would be faster. Use warn=False to silence this warning.
  warnings.warn(


Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('standardscaler', StandardScaler())])),
                ('ridgecv', RidgeCV(alphas=[0.01]))])

In [12]:
scores = pipeline.predict(X_test)

In [13]:
scores

array([[-5.0319661e-03,  2.9659900e-04, -6.7416352e-04, ...,
        -4.0480003e-02,  1.7629522e-01,  4.2345423e-02],
       [-8.7825395e-03,  5.1766896e-04, -1.1766497e-03, ...,
         2.5670868e-01, -1.5915927e-01, -3.4860995e-02],
       [-7.4275406e-03,  4.3780133e-04, -9.9511305e-04, ...,
         1.1837522e-01, -2.4794486e-01, -6.7438826e-02],
       ...,
       [ 4.4152015e-03, -2.6024497e-04,  5.9153244e-04, ...,
         2.1377805e-01, -3.0304834e-01, -1.6260592e-02],
       [-1.0169807e-02,  5.9943856e-04, -1.3625105e-03, ...,
        -1.5578383e-01,  1.4210662e-01,  1.0523617e-01],
       [-2.5349068e-03,  1.4941476e-04, -3.3961656e-04, ...,
        -7.8510955e-02,  1.2246935e-01,  9.2457920e-02]], dtype=float32)

In [ ]:
np.save(f'{savedir}/{subject}_{"_".join(roi)}_scores_{target}.npy',scores)

____
____